In [5]:
import sqlite3
import pandas as pd
from datetime import datetime

In [6]:
def insert_unique_data_to_db(df, table_name, conn, cursor):
    """
    Wstawia unikalne dane do bazy danych, sprawdzając, czy rekord już istnieje w tabeli.

    Args:
        df (DataFrame): DataFrame z danymi do wstawienia.
        table_name (str): Nazwa tabeli, do której dane mają być wstawione.
        conn (sqlite3.Connection): Obiekt połączenia z bazą danych SQLite.
        cursor (sqlite3.Cursor): Obiekt kursora SQLite.
    """
    df.columns = df.columns.str.replace(' ', '_').str.replace('ł', 'l').str.replace('ż', 'z')
    df['data'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    for _, row in df.iterrows():
        query_check = f'''
        SELECT 1 FROM {table_name}
        WHERE Cena = ? AND Cena_za_m2 = ? AND Lokalizacja = ? AND URL = ?
        '''
        cursor.execute(query_check, (row['Cena'], row['Cena_za_m2'], row['Lokalizacja'], row['URL']))
        exists = cursor.fetchone()

        if not exists:  
            query_insert = f'''
            INSERT INTO {table_name} (
                Cena, Cena_za_m2, Lokalizacja, Powierzchnia_calkowita, Liczba_pokoi, Piętro, Typ_budynku, Rok_budowy,
                Typ_ogloszeniodawcy, URL, Forma_wlasnosci, Dostepne_od, Stan_wykonczenia, Rynek, winda, ogrzewanie,
                wojewodztwo, miasto, dzielnica, ulica, Liczba_pieter, balkon, piwnica, garaz, miejsce_parkingowe,
                Ogrzewanie_rodzaj, ochrona, Ogłoszeniodawca, dzielnica_num, data
            ) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);
            '''
            cursor.execute(query_insert, tuple(row))
    conn.commit()
    print(f"Dane zostały wstawione do tabeli '{table_name}'.")



In [9]:
import sqlite3
import pandas as pd
from datetime import datetime

#df = pd.read_excel(r'C:\Users\PC\Desktop\scrapping_danych\Data_all_preprocessed_actual.xlsx') 
df = pd.read_excel(r'Data_all_preprocessed_actual.xlsx') 
#df_arch = pd.read_excel(rf'Data_all_preprocessed_.xlsx')  

df['Cena'] = df['Cena'].astype(float)
df['Cena za m2'] = df['Cena za m2'].astype(float)
df['Powierzchnia całkowita'] = df['Powierzchnia całkowita'].astype(float)
df['Liczba pokoi'] = df['Liczba pokoi'].astype(int)
df['Liczba pięter'] = pd.to_numeric(df['Liczba pięter'], errors='coerce') 
df['Piętro'] = pd.to_numeric(df['Piętro'], errors='coerce')


df.fillna('brak informacji', inplace=True)

conn = sqlite3.connect('django\realestate\real_estate2.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS aktualne (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Cena FLOAT,
    Cena_za_m2 FLOAT,
    Lokalizacja TEXT,
    Powierzchnia_calkowita FLOAT,
    Liczba_pokoi INTEGER,
    Piętro FLOAT,
    Typ_budynku TEXT,
    Rok_budowy FLOAT,
    Typ_ogloszeniodawcy TEXT,
    URL TEXT,
    Forma_wlasnosci TEXT,
    Dostepne_od TEXT,
    Stan_wykonczenia TEXT,
    Rynek TEXT,
    winda INTEGER,
    ogrzewanie INTEGER,
    wojewodztwo TEXT,
    miasto TEXT,
    dzielnica TEXT,
    ulica TEXT,
    Liczba_pieter FLOAT,
    balkon INTEGER,
    piwnica INTEGER,
    garaz INTEGER,
    miejsce_parkingowe INTEGER,
    Ogrzewanie_rodzaj TEXT,
    ochrona INTEGER,
    Ogłoszeniodawca TEXT,
    dzielnica_num INTEGER,
    data TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS archiwalne (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Cena FLOAT,
    Cena_za_m2 FLOAT,
    Lokalizacja TEXT,
    Powierzchnia_calkowita FLOAT,
    Liczba_pokoi INTEGER,
    Piętro FLOAT,
    Typ_budynku TEXT,
    Rok_budowy FLOAT,
    Typ_ogloszeniodawcy TEXT,
    URL TEXT,
    Forma_wlasnosci TEXT,
    Dostepne_od TEXT,
    Stan_wykonczenia TEXT,
    Rynek TEXT,
    winda INTEGER,
    ogrzewanie INTEGER,
    wojewodztwo TEXT,
    miasto TEXT,
    dzielnica TEXT,
    ulica TEXT,
    Liczba_pieter FLOAT,
    balkon INTEGER,
    piwnica INTEGER,
    garaz INTEGER,
    miejsce_parkingowe INTEGER,
    Ogrzewanie_rodzaj TEXT,
    ochrona INTEGER,
    Ogłoszeniodawca TEXT,
    dzielnica_num INTEGER,
    data TEXT
)
''')

def insert_data_to_db(df, table_name):
    df['data'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    records = df.to_records(index=False)
    data = [tuple(record) for record in records]

    query = f'''
    INSERT INTO {table_name} (
        Cena, Cena_za_m2, Lokalizacja, Powierzchnia_calkowita, Liczba_pokoi, Piętro, Typ_budynku, Rok_budowy,
        Typ_ogloszeniodawcy, URL, Forma_wlasnosci, Dostepne_od, Stan_wykonczenia, Rynek, winda, ogrzewanie, wojewodztwo,
        miasto, dzielnica, ulica, Liczba_pieter, balkon, piwnica, garaz, miejsce_parkingowe, Ogrzewanie_rodzaj,
        ochrona, Ogłoszeniodawca, dzielnica_num, data
    ) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);
    '''
    cursor.executemany(query, data)
    conn.commit()

def clear_table(table_name):
    query = f"DELETE FROM {table_name};"
    cursor.execute(query)
    conn.commit()
    print(f"Zawartość tabeli '{table_name}' została usunięta.")

clear_table('aktualne')
df.columns = df.columns.str.replace(' ', '_').str.replace('ł', 'l').str.replace('ż', 'z')
df.to_sql('aktualne', conn, if_exists='replace', index=False)
insert_unique_data_to_db(df, 'archiwalne', conn, cursor)
#df_arch.columns = df_arch.columns.str.replace(' ', '_').str.replace('ł', 'l').str.replace('ż', 'z')
#df_arch.to_sql('archiwalne', conn, if_exists='replace', index=False)
#insert_data_to_db(df, 'aktualne')
# insert_data_to_db(df_archiwalne, 'archiwalne')
conn.close()

print("Dane zapisane do bazy.")


C:\Users\kaboo\AppData\Local\Temp\ipykernel_26252\3383623800.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'brak informacji' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('brak informacji', inplace=True)


Zawartość tabeli 'aktualne' została usunięta.
Dane zostały wstawione do tabeli 'archiwalne'.
Dane zapisane do bazy.


In [11]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('real_estate2.db')
query = """SELECT * from aktualne
"""
df_from_db = pd.read_sql(query, conn)
conn.close()

df_from_db

,Cena,Cena_za_m2,Lokalizacja,Powierzchnia_calkowita,Liczba_pokoi,Piętro,Typ_budynku,Rok_budowy,Typ_ogloszeniodawcy,URL,...,ulica,Liczba_pięter,balkon,piwnica,garaz,miejsce_parkingowe,Ogrzewanie_rodzaj,ochrona,Ogloszeniodawca,dzielnica_num
0,570780.0,10659.00,"ul. Złota, Dąb, Katowice, śląskie",53.55,3,2.0,blok,2024.0,biuro nieruchomości,https://www.otodom.pl/pl/oferta/3-pokoje-balko...,...,ul. Złota,5.0,0,0,1,1,miejskie,0,brak informacji,3
1,643000.0,6801.00,"ul. Wojewódzka, Śródmieście, Katowice, śląskie",94.55,3,2.0,kamienica,1904.0,biuro nieruchomości,https://www.otodom.pl/pl/oferta/przestronne-kl...,...,ul. Wojewódzka,4.0,0,0,0,0,miejskie,0,brak informacji,1
2,650000.0,16667.00,"ul. Meteorologów, Brynów-Osiedle Zgrzebnioka, ...",39.00,2,3.0,apartamentowiec,2023.0,biuro nieruchomości,https://www.otodom.pl/pl/oferta/nowoczesny-apa...,...,ul. Meteorologów,4.0,1,0,1,1,miejskie,1,brak informacji,3
3,400000.0,8325.00,"Ligota-Panewniki, Katowice, śląskie",48.05,2,0.0,blok,1960.0,prywatny,https://www.otodom.pl/pl/oferta/sloneczne-mies...,...,brak informacji,2.0,0,1,1,1,gazowe,0,brak informacji,3
4,724941.0,12950.00,"ul. gen. Kazimierza Pułaskiego 23, Osiedle Pad...",55.98,2,6.0,blok,2026.0,deweloper,https://www.otodom.pl/pl/oferta/panorama-3-sta...,...,ul. gen. Kazimierza Pułaskiego 23,8.0,1,0,1,1,brak informacji,1,brak informacji,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3346,571900.0,9615.00,"śląskie,Katowice,Koszutka",59.48,3,2.0,park,2023.0,brak informacji,https://www.domiporta.pl/nieruchomosci/sprzeda...,...,brak informacji,4.0,0,0,0,1,brak informacji,0,Dayton Nieruchomości,1
3347,969000.0,17108.05,"śląskie,Katowice,Adama Mickiewicza",56.64,3,20.0,apartamentowiec,brak informacji,brak informacji,https://www.domiporta.pl/nieruchomosci/sprzeda...,...,Adama Mickiewicza,27.0,0,0,0,0,brak informacji,0,NewState REI,1
3348,560000.0,10980.39,"śląskie,Katowice,Śródmieście",51.00,2,1.0,kamienica,brak informacji,brak informacji,https://www.domiporta.pl/nieruchomosci/sprzeda...,...,brak informacji,4.0,0,1,0,0,brak informacji,0,ESTATEPRO NIERUCHOMOŚCI,1
3349,399000.0,9307.21,"śląskie,Katowice,Dolina Trzech Stawów,gen. Józ...",42.87,2,10.0,brak informacji,1986.0,brak informacji,https://www.domiporta.pl/nieruchomosci/sprzeda...,...,gen. Józefa Sowińskiego,10.0,0,0,0,0,brak informacji,0,Fiesta Nieruchomości,2


In [41]:
import sqlite3
import requests
import pandas as pd
from bs4 import BeautifulSoup

def check_page_status(url):
    """Sprawdza status strony i obecność komunikatu o nieaktualności."""
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        if response.status_code != 200:
            print(f"Strona niedostępna: {url}")
            return False
        soup = BeautifulSoup(response.text, 'html.parser')

        domiporta_message = soup.find('div', class_='archive__info')
        if domiporta_message and soup.find('h3', class_='archive__title'):
            print(f"Ogłoszenie nieaktualne w Domiporta: {url}") 
            return False
        otodom_message = soup.find('div', {'data-cy': 'expired-ad-alert'})
        if otodom_message and soup.find('span', class_='css-5ujp2z'):
            print(f"Ogłoszenie nieaktualne w Otodom: {url}")
            return False

        print(f"Ogłoszenie aktualne: {url}")
        return True

    except Exception as e:
        print(f"Błąd podczas sprawdzania strony {url}: {e}")
        return False


In [24]:
'''
df_from_db['URL_exists'] = df_from_db['URL'].apply(check_page_status)
non_existent_urls = df_from_db[df_from_db['URL_exists'] == False]
print("URL, które nie istnieją:")
print(non_existent_urls[['id', 'URL']])
'''


Strona niedostępna: https://www.otodom.pl/pl/oferta/centrum-katowice-zabrska-mickiewicza-22-pietro-ID4tbuR
Ogłoszenie aktualne: https://www.otodom.pl/pl/oferta/medykow-32-mieszkanie-3-pok-84-ID4nj2X
Ogłoszenie aktualne: https://www.domiporta.pl/nieruchomosci/sprzedam-mieszkanie-trzypokojowe-katowice-koszutka-jesionowa-94m2/154975177
Ogłoszenie aktualne: https://www.domiporta.pl/nieruchomosci/sprzedam-mieszkanie-dwupokojowe-katowice-brynow-mikolowska-50m2/155227276
Ogłoszenie aktualne: https://www.otodom.pl/pl/oferta/przestronne-m360-21m2burowiec-ID4tt0R
Strona niedostępna: https://www.otodom.pl/pl/oferta/mieszkanie-50-55-m-katowice-ID4t81T
Ogłoszenie aktualne: https://www.otodom.pl/pl/oferta/panorama-3-stawy-mieszkanie-2-pok-4-b34-ID4nVCG
Ogłoszenie aktualne: https://www.otodom.pl/pl/oferta/2-pokojowe-mieszkanie-39m2-loggia-ID4pFtn
Ogłoszenie aktualne: https://www.otodom.pl/pl/oferta/medykow-32-mieszkanie-4-pok-55-ID4nj2n
Ogłoszenie nieaktualne w Domiporta: https://www.domiporta.pl/nie

In [ ]:
'''
ids_to_remove = non_existent_urls['id'].tolist()

if ids_to_remove:
    placeholders = ', '.join('?' for _ in ids_to_remove)
    query = f"DELETE FROM your_table_name WHERE id IN ({placeholders})"
    cursor.execute(query, ids_to_remove)
    conn.commit()
    print(f"Usunięto {cursor.rowcount} rekordów z bazy danych.")
else:
    print("Brak nieistniejących URL do usunięcia.")
'''

In [3]:
import sqlite3

conn = sqlite3.connect(r'C:\Users\kaboo\OneDrive\Pulpit\Estate_scrapper_model_analyse\real_estate2.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM aktualne")  # Zmień na właściwą nazwę tabeli
rows = cursor.fetchall()


[(326000.0, 7143.0, 'ul. Jaworowa, Piotrowice-Ochojec, Katowice, śląskie', 45.64, 2, '1.0', 'blok', '1964.0', 'biuro nieruchomości', 'https://www.otodom.pl/pl/oferta/2-pokoje-balkon-zamkniete-osiedle-piotrowice-ID4s163', 'brak informacji', 'brak informacji', 'do remontu', 'wtórny', 0, 1, 'śląskie', 'Katowice', 'Piotrowice-Ochojec', 'ul. Jaworowa', '4.0', 1, 1, 0, 0, 'miejskie', 0, 'brak informacji', 5), (1220000.0, 15062.0, 'ul. gen. Władysława Sikorskiego, Osiedle Paderewskiego-Muchowiec, Katowice, śląskie', 81.0, 3, '3.0', 'apartamentowiec', '2014.0', 'biuro nieruchomości', 'https://www.otodom.pl/pl/oferta/apartament-w-dolinie-trzech-stawow-sikorskiego-ID4pNl3', 'pełna własność', 'brak informacji', 'do zamieszkania', 'wtórny', 1, 1, 'śląskie', 'Katowice', 'Os. Paderewskiego – Muchowiec', 'ul. gen. Władysława Sikorskiego', '7.0', 1, 0, 1, 1, 'miejskie', 1, 'brak informacji', 2), (777000.0, 9107.0, 'ul. Juliusza Słowackiego, Śródmieście, Katowice, śląskie', 85.32, 2, '5.0', 'kamienica'

In [4]:
rows

[(326000.0,
  7143.0,
  'ul. Jaworowa, Piotrowice-Ochojec, Katowice, śląskie',
  45.64,
  2,
  '1.0',
  'blok',
  '1964.0',
  'biuro nieruchomości',
  'https://www.otodom.pl/pl/oferta/2-pokoje-balkon-zamkniete-osiedle-piotrowice-ID4s163',
  'brak informacji',
  'brak informacji',
  'do remontu',
  'wtórny',
  0,
  1,
  'śląskie',
  'Katowice',
  'Piotrowice-Ochojec',
  'ul. Jaworowa',
  '4.0',
  1,
  1,
  0,
  0,
  'miejskie',
  0,
  'brak informacji',
  5),
 (1220000.0,
  15062.0,
  'ul. gen. Władysława Sikorskiego, Osiedle Paderewskiego-Muchowiec, Katowice, śląskie',
  81.0,
  3,
  '3.0',
  'apartamentowiec',
  '2014.0',
  'biuro nieruchomości',
  'https://www.otodom.pl/pl/oferta/apartament-w-dolinie-trzech-stawow-sikorskiego-ID4pNl3',
  'pełna własność',
  'brak informacji',
  'do zamieszkania',
  'wtórny',
  1,
  1,
  'śląskie',
  'Katowice',
  'Os. Paderewskiego – Muchowiec',
  'ul. gen. Władysława Sikorskiego',
  '7.0',
  1,
  0,
  1,
  1,
  'miejskie',
  1,
  'brak informacji',
